In [1]:
import pathlib

import awkward as ak
import librosa
from loguru import logger
import numpy as np
import pandas as pd
import scipy
import torch
from tqdm import tqdm

from utils.utils import import_audio_dataset, file_names_map, encode_file_names, \
    pad_awkward_array, audio_tokenizer, downsample_tensor, get_file_list

In [2]:
Xr = import_audio_dataset('data/ravdess/', '.wav')
# Xc = import_audio_dataset('data/crema/', '.wav')

2024-03-19 10:01:15.196 | INFO     | utils.utils:import_audio_dataset:71 - Loading 2452 files to list...
  0%|          | 0/2452 [00:00<?, ?it/s]

100%|██████████| 2452/2452 [00:19<00:00, 124.10it/s]
2024-03-19 10:01:34.964 | INFO     | utils.utils:import_audio_dataset:78 - Loading 2452 files to awkward array...
100%|██████████| 2452/2452 [07:19<00:00,  6.75it/s]

: 

In [ ]:
Yr = encode_file_names(get_file_list('data/ravdess/', '.wav'), 'ravdess')
Yc = encode_file_names(get_file_list('data/crema/', '.wav'), 'crema')